In [3]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
#from visual import *
import numpy as np
import timeit
from math import *

In [4]:
def initialize(speed = 1, N = 100, width = 1000, height = 1000):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    agents = [Point(width*random(), height*random()) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    for i in range(N):
        theta = 2 * np.pi * random()
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)

    return agents, speeds, draw_agents(agents, width, height)


def draw_agents(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
            weightedSpeed = a * speeds[nearest_neighbours[i]]
            speeds[i] = speeds[i] + weightedSpeed
            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------


def get_distances(agent, agents):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    
    dists = [(a.getX() - agent.getX())**2 + (a.getY() - agent.getY())**2 for a in agents]
    for i in range(len(dists)):
        if dists[i] == 0:
            dists[i] = 0.1
    return dists


def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def treat_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return

In [23]:
def couzin(agents, speeds, N, s, noise, rr=1, ro=2, ra=3):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue

            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            
            elif not repulsion_flag:

                if distances[j] < ro:
                    o_dir = o_dir + speeds[j]
                
                elif distances[j] < ra:
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
            
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = o_dir + a_dir
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir 

def gbias(speeds, N, s, prop, bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * bias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir

def accuracy(speeds, N, bias):
    dev = 0.0
    for i in range(N):
        dev += np.arccos(np.dot(bias,normalized(speeds[i])))
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
def elongation(agents, N, window, point_cm):
    #cm = np.array([0.0, 0.0])
    # Pode melhorar com 1 for(i)
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    cm = np.mean(poses,axis=0)
    std = np.std(poses,axis=0)
    
    point_cm = Point(cm[0],cm[1])
    point_cm.draw(window)
    point_cm.setFill("red")
    print (cm, std)
#    for i in range(N):
 #       x_dev = 
        
                         
    
def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir

def gueron():
    return

In [24]:
def simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    point_cm=Point(0,0)
    point_cm.setFill("red")
    agents, speeds, window = initialize(s, N, width, height)
    for i in range(N_steps):
        next_step(agents, speeds, dt, N)
        import timeit

        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        couzin(agents, speeds, N, s, noise,100,1900,2000)
        gbias(speeds, N, s, prop, bias, weight)
        accuracy(speeds, N, bias)
        
        
        #point_cm.undraw()
        elongation(agents,N,  window, point_cm)
        
        ## BOUNDARY CONDITIONS
        #treat_boundary(width, height, agents, speeds, N)
        periodic_boundary(width, height, agents, speeds, N)
        #print(v_avg)
        #v_avg = get_v_avg(speeds)
        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    

#simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias)
simulate(3000, 0.1, 1, 50, 500, 500, 2, 0.02, 0.4,0.5, np.array([0.0,-1.0]))

(array([ 260.9298986 ,  282.77526928]), array([ 149.72690152,  152.88998027]))
(array([ 260.77907326,  283.04713612]), array([ 149.53892197,  153.27147848]))
(array([ 260.47212757,  283.12322244]), array([ 149.42093257,  153.6463854 ]))
(array([ 259.97816786,  283.46872496]), array([ 149.76819064,  154.00715182]))
(array([ 259.74322087,  283.47099209]), array([ 150.48183498,  154.45078925]))
(array([ 109.62496157,  283.58963591]), array([ 278.52178331,  154.70294494]))
(array([ 259.67117088,  243.58303848]), array([ 151.37268775,  173.89785307]))
(array([ 259.58263966,  273.76192403]), array([ 151.59970781,  156.92944378]))
(array([ 259.64046633,  274.12396863]), array([ 152.07393765,  156.99720279]))
(array([ 259.77781703,  274.02897023]), array([ 152.4058371 ,  156.70878047]))
(array([ 259.88917814,  274.09027791]), array([ 153.08494934,  156.73177366]))
(array([ 260.10063417,  274.01361693]), array([ 153.47922068,  156.89131827]))
(array([ 260.28137115,  273.78429205]), array([ 154.

(array([ 245.14140727,  223.77560084]), array([ 163.21132481,  167.04040574]))
(array([ 244.92110259,  392.85639076]), array([ 163.97473435,  404.17437835]))
(array([ 244.8512578 ,  242.38876009]), array([ 164.32580542,  216.86660446]))
(array([ 244.6599389 ,  551.56748153]), array([ 164.99563453,  264.94104349]))
(array([ 244.69653818,  210.60002838]), array([ 165.41799448,  192.22475012]))
(array([ 244.38383475,  319.83041205]), array([ 166.25233947,  217.10060107]))
(array([ 244.2689159 ,  218.85184039]), array([ 166.69447079,  166.15816366]))
(array([ 244.27904174,  237.60561395]), array([ 167.4336708,  151.1079794]))
(array([ 244.21216373,  266.46541132]), array([ 168.16165634,  193.15277209]))
(array([ 244.16570337,  235.33199531]), array([ 168.80549551,  151.30124892]))
(array([ 244.0748746 ,  343.99828466]), array([ 169.42599159,  240.1721509 ]))
(array([ 243.82243452,  242.78400592]), array([ 169.98960965,  152.12727915]))
(array([ 243.852117 ,  281.4946069]), array([ 170.5118

(array([ 186.05525868,  277.83617231]), array([ 135.456246  ,  157.58356746]))
(array([ 186.2251796 ,  276.07628956]), array([ 135.28569591,  157.42268576]))
(array([ 186.39797164,  274.34207145]), array([ 135.20557844,  157.43016097]))
(array([-313.3961132 ,  272.65889027]), array([ 135.00110511,  157.2916295 ]))
(array([ 186.86889379,  271.06581186]), array([ 134.8919785 ,  157.19096103]))
(array([-312.9620759 ,  269.27036115]), array([ 134.77289035,  157.07222968]))
(array([ 177.19532856,  477.57356481]), array([ 129.22830169,  282.79118202]))
(array([ 177.32099464,  276.60356345]), array([ 129.13007009,  155.11256071]))
(array([ 347.38480407,  275.21100687]), array([ 259.47728414,  155.28149412]))
(array([ 187.45648602,  273.7436153 ]), array([ 134.25270783,  155.01171499]))
(array([ 187.74516014,  272.25649949]), array([ 134.48152652,  154.93329481]))
(array([ 188.01792429,  270.74373618]), array([ 134.67998002,  154.90724544]))
(array([ 188.08692985,   59.19429827]), array([ 134.

(array([ 221.32676774,  233.16710501]), array([ 141.57744548,  131.29391149]))
(array([ 221.16902722,  231.50268436]), array([ 141.22025337,  131.56228678]))
(array([ 220.98788073,  229.7801838 ]), array([ 140.93598523,  131.65178574]))
(array([ 220.61681872,  228.05988838]), array([ 140.65579416,  131.85132842]))
(array([ 220.2460982 ,  226.41977075]), array([ 140.3323274 ,  131.87183908]))
(array([ 219.92743602,  224.74492219]), array([ 139.99410679,  132.13776372]))
(array([ 219.64192625,  222.89997459]), array([ 139.82378897,  132.31991358]))
(array([ 219.319942  ,  221.16567779]), array([ 139.63347858,  132.61226175]))
(array([ 218.96015319,  219.37484916]), array([ 139.33514722,  132.91312054]))
(array([ 218.86588546,  217.74444756]), array([ 139.24902959,  133.46616629]))
(array([ 218.66678915,  216.08523203]), array([ 139.01770914,  133.74933216]))
(array([ 218.44027116,  214.36029787]), array([ 138.91784324,  134.10317016]))
(array([ 218.45612286,  212.51735084]), array([ 138.

(array([ 218.68926734,  294.62566471]), array([ 134.73052926,  160.81546855]))
(array([ 219.01391824,  292.75468387]), array([ 134.87787209,  160.81212303]))
(array([ 219.39568194,  290.92938272]), array([ 134.97505661,  160.83980544]))
(array([ 219.8264726 ,  289.03574002]), array([ 135.04587933,  160.86512313]))
(array([ 220.16146249,  287.13195625]), array([ 135.15201888,  160.87238296]))
(array([ 220.44565385,  285.24367259]), array([ 135.2398514 ,  160.88598016]))
(array([ 220.7289324 ,  693.30054874]), array([ 135.29776009,  270.2776257 ]))
(array([ 220.83699213,  291.49862082]), array([ 135.32073472,  158.18451142]))
(array([ 221.02578597,  289.58183432]), array([ 135.45110599,  158.18387985]))
(array([ 221.27489036,  287.61729514]), array([ 135.50538785,  158.19017532]))
(array([ 221.45122522,  555.68662137]), array([ 135.63969111,  295.97033155]))
(array([ 221.60307847,  294.13585682]), array([ 135.47481112,  155.17503854]))
(array([ 221.92776068,  292.25995137]), array([ 135.

...

(array([ 258.65138002,  260.1744771 ]), array([ 115.71510587,  158.95427382]))
(array([ 258.39783908,  258.78978899]), array([ 115.8396695 ,  158.49359937]))
(array([ 258.01183376,  257.48191772]), array([ 116.05577782,  158.11903337]))
(array([ 257.6662123 ,  256.16917234]), array([ 116.28198445,  157.76614289]))
(array([ 257.28256123,  255.02927692]), array([ 116.47967418,  157.64842286]))
(array([ 256.82177808,  253.69175815]), array([ 116.70131009,  157.28522353]))
(array([ 256.36869291,  252.38515094]), array([ 116.93547677,  156.98781984]))
(array([ 255.91654148,  251.07926693]), array([ 117.17453668,  156.6940027 ]))
(array([ 255.5365955 ,  249.88476195]), array([ 117.36546918,  156.37116257]))
(array([ 255.21835849,  248.64032165]), array([ 117.5949968 ,  156.14535409]))
(array([ 254.82174732,  247.33996319]), array([ 117.88498748,  155.84200641]))
(array([ 254.49929687,  746.22061365]), array([ 118.20907443,  155.79753049]))
(array([ 254.06167957,  254.75393085]), array([ 118.

(array([ 230.46409579,  236.05643764]), array([ 139.98088251,  152.53952435]))
(array([ 230.32603551,  234.97530984]), array([ 139.85787744,  152.24561433]))
(array([ 230.35608346,  373.98993661]), array([ 139.99734245,  249.00831013]))
(array([ 230.34221963,  243.03013599]), array([ 139.9803707 ,  151.72060034]))
(array([ 230.21920634,  242.07852809]), array([ 139.77617483,  151.4041833 ]))
(array([ 230.19015574,  240.90521673]), array([ 139.74951498,  151.05384392]))
(array([ 230.18088545,  419.94160164]), array([ 139.72577585,  266.65305221]))
(array([ 230.16507728,  248.9235011 ]), array([ 139.74204523,  150.28057396]))
(array([ 230.26342023,  248.06413794]), array([ 139.87485136,  149.7490364 ]))
(array([ 230.28367289,  247.10565303]), array([ 139.80475263,  149.29012838]))
(array([ 230.34711964,  695.97508594]), array([ 139.90091054,  215.9280912 ]))
(array([ 230.38181686,  255.09461031]), array([ 140.01891762,  148.08346522]))
(array([ 230.56219291,  254.11385414]), array([ 140.

(array([ 227.77580577,  233.55565439]), array([ 154.0506251 ,  151.92581729]))
(array([ 217.71424084,  232.66325489]), array([ 265.65690357,  152.4674604 ]))
(array([ 217.68317903,  231.77927703]), array([ 182.72140757,  153.21546322]))
(array([ 287.33820641,  230.81198132]), array([ 208.26386936,  153.88189481]))
(array([ 227.04548941,  229.7833173 ]), array([ 155.20885732,  154.38529447]))
(array([ 306.59077763,  228.71921672]), array([ 204.02185427,  154.76303889]))
(array([ 176.22527883,  227.72463395]), array([ 229.00626339,  155.43387165]))
(array([ 305.96927025,  256.60669787]), array([ 204.3822729 ,  179.40766196]))
(array([ 245.70546517,  235.49695909]), array([ 158.16638224,  157.91887498]))
(array([ 245.32648417,  234.19951058]), array([ 158.68822812,  158.3905987 ]))
(array([ 684.96965432,  522.94592257]), array([ 202.92496203,  260.50229866]))
(array([ 254.54302354,  241.84840999]), array([ 159.50237364,  160.01225271]))
(array([ 414.17877335,  240.81617575]), array([ 256.

(array([  37.46482196,  241.11426928]), array([ 288.90523269,  149.95882914]))
(array([ 257.15998599,  479.25335368]), array([ 143.94130277,  334.81486017]))
(array([ 256.90899355,  247.46957038]), array([ 144.11517044,  150.09291916]))
(array([ 256.78540841,  245.66204337]), array([ 144.16906895,  150.12575015]))
(array([ 256.64903828,  243.82926108]), array([ 144.14476311,  150.18959648]))
(array([ 256.40833752,  242.01221497]), array([ 144.21378828,  150.26061598]))
(array([ 256.17439525,  240.17358172]), array([ 144.06687695,  150.35791909]))
(array([ 255.97397614,  238.34485022]), array([ 144.01056575,  150.32623681]))
(array([ 255.66495568,  236.44689564]), array([ 144.03525519,  150.2999264 ]))
(array([ 255.39088703,  234.66295107]), array([ 144.08985558,  150.50156469]))
(array([ 255.17610538,  232.78122264]), array([ 144.03084586,  150.56037813]))
(array([ 254.98920591,  230.97545629]), array([ 144.01943286,  150.65620748]))
(array([ 254.70055576,  229.03182588]), array([ 143.

(array([ 298.00006412,  262.45502804]), array([ 146.13006963,  104.39602332]))
(array([ 298.80799349,  260.96395717]), array([ 146.31516086,  104.24609454]))
(array([ 299.48277053,  259.56654494]), array([ 146.60348179,  103.9644564 ]))
(array([ 300.27519897,  258.13982074]), array([ 146.75743846,  103.7720199 ]))
(array([ 301.01403538,  256.70262342]), array([ 146.99585332,  103.55276629]))
(array([ 301.77671574,  255.14768404]), array([ 147.15973658,  103.36735472]))
(array([ 302.61430386,  253.61726179]), array([ 147.38000065,  103.28147235]))
(array([-136.65635312,  252.10753961]), array([ 226.24852137,  103.13282806]))
(array([ 294.13957275,  250.64935311]), array([ 150.81599203,  103.11342932]))
(array([ 295.05482326,  249.17887426]), array([ 150.9496772 ,  102.89065513]))
(array([ 295.780421  ,  247.60274597]), array([ 151.17560134,  102.83472843]))
(array([ 296.53484963,  246.01441724]), array([ 151.29024262,  102.79631719]))
(array([ 297.41472047,  244.49438766]), array([ 151.